Plotting the mean probabilities of all 10 runs for each distribution according to the best individual

In [25]:
import os
import re
import pandas as pd
import numpy as np
import pickle as pkl
from experiments import get_state
from itertools import product
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from experiments.util import load_circuit
from sanchez_ansatz import SanchezAnsatz
import matplotlib.pyplot as plt

num_qubits = 15
eps = 0.05

RESULTS_DIR = "results/genetic/distributions_full_training"
x_operations = ["cycle", "order", "pmx"]
mut_operations = ["insert", "inverse", "scramble"]

#### $log$-Normal $(\mu = 0.1, \sigma = 0.3)$

In [30]:
def get_statevector(ansatz: QuantumCircuit):
    sv_sim = Aer.get_backend("statevector_simulator")
    job = sv_sim.run(ansatz)
    result = job.result()
    statevector = result.get_statevector()
    return statevector.data

def get_best_individual(inividual_path: str): 
    df = pd.read_csv(inividual_path, header=None)
    digits = re.findall(r"\d+", df.iloc[-1].values[0])
    digits = [int(d) for d in digits]
    return digits

In [ ]:
x_points = (0, 1)
mu = .1
sig = .3

target_state = get_state(num_qubits, state_type="lognormal", state_params={"x_points": x_points, "s": 1, "loc": mu, "scale": sig})

In [38]:
n_runs = 10
num_qubits = 15
eps = 0.05
dist_name = "lognormal"
x_op = "order"
mut_op = "scramble"

exp_dir = os.path.join(RESULTS_DIR, f"{dist_name}_{x_op}_{mut_op}")

o_ansatz = SanchezAnsatz(target_state=target_state, eps=eps)

m_ansatz = SanchezAnsatz(target_state=target_state, eps=eps, build_modified=True)

original_outputs = []
modified_outputs = []

for run_idx in range(1, 3): 

    # run directory
    run_dir = os.path.join(exp_dir, f"run_{run_idx}")

    # loading best individuals
    best_individuals_original_path = os.path.join(run_dir, "csv", f"original_best_individual_{num_qubits}qb_{eps}eps.csv")
    best_individual_original = get_best_individual(best_individuals_original_path)

    t_o_ansatz = transpile(o_ansatz, basis_gates=["cx", "u"], initial_layout=best_individual_original)

    best_individuals_modified_path = os.path.join(run_dir, "csv", f"modified_best_individual_{num_qubits}qb_{eps}eps.csv")
    best_individual_modified = get_best_individual(best_individuals_modified_path)

    t_m_ansatz = transpile(m_ansatz, basis_gates=["cx", "u"], initial_layout=best_individual_modified)


    # loading best individuals parameters
    best_params_original_path = os.path.join(run_dir, "csv", f"original_best_individual_params_{num_qubits}qb_{eps}eps.csv")
    best_params_original = np.loadtxt(best_params_original_path, delimiter=",")

    best_params_modified_path = os.path.join(run_dir, "csv", f"modified_best_individual_params_{num_qubits}qb_{eps}eps.csv")
    best_params_modified = np.loadtxt(best_params_modified_path, delimiter=",")

    # running simulations
    try: 
        t_o_ansatz_params = t_o_ansatz.assign_parameters(best_params_original[-1])
        t_m_ansatz_params = t_m_ansatz.assign_parameters(best_params_modified[-1])
    except: 
        print("Error")

    # getting outputs
    o_output = get_statevector(t_o_ansatz_params)
    m_output = get_statevector(t_m_ansatz_params)

    original_outputs.append(o_output)
    modified_outputs.append(m_output)

original_outputs = np.array(original_outputs)
modified_outputs = np.array(modified_outputs)

In [29]:
path = f"{RESULTS_DIR}/lognormal_order_scramble/run_1/csv/original_best_individual_15qb_0.05eps.csv"

df = pd.read_csv(path, header=None)
digits = re.findall(r"\d+", df.iloc[-1].values[0])
print("Strings: ", digits)
digits = [int(d) for d in digits]
print("Integers: ", digits)

Strings:  ['14', '8', '2', '5', '3', '1', '4', '7', '10', '11', '6', '9', '13', '0', '12']
Integers:  [14, 8, 2, 5, 3, 1, 4, 7, 10, 11, 6, 9, 13, 0, 12]


In [ ]:

freqs = np.abs(state)**2

x = np.linspace(*x_points, num=2**num_qubits)

plt.plot(x, freqs, label="target")
plt.legend(loc="best")
plt.ylabel("P(X)")
plt.xlabel("X")
